In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pycm livelossplot
from livelossplot import PlotLosses
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] ='0'
import torch  # Pytorch
import torch.nn as nn  # Neural network module
import torch.nn.functional as fn  # Function module
import random
from torch.utils.data import Dataset, DataLoader, TensorDataset,Subset
from torchvision import datasets  # Datasets from torchvision
from torchvision import transforms  # Transforms from torchvision

import matplotlib.pyplot as plt  # Plotting using matplotlib
import numpy as np  # Numpy

from PIL import Image

import os
from os import listdir

device = 'cuda'  # Set out device to GPU

print('done')  # Let me know this cell has finished

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.7/595.7 KB 18.4 MB/s eta 0:00:00
done


In [4]:
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True


def set_device(device="cpu", idx=0):
    if device != "cpu":
        if torch.cuda.device_count() > idx and torch.cuda.is_available():
            print("Cuda installed! Running on GPU {} {}!".format(idx, torch.cuda.get_device_name(idx)))
            device="cuda:{}".format(idx)
        elif torch.cuda.device_count() > 0 and torch.cuda.is_available():
            print("Cuda installed but only {} GPU(s) available! Running on GPU 0 {}!".format(torch.cuda.device_count(), torch.cuda.get_device_name()))
            device="cuda:0"
        else:
            device="cpu"
            print("No GPU available! Running on CPU")
    return device

device = set_device("cuda")

No GPU available! Running on CPU


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [7]:
import pandas as pd
#creating dataset
images_path = "drive/MyDrive/Mars_GAN/images"
labels_path = "drive/MyDrive/Mars_GAN/labels"

image_list = []
label_list = []

for labelname in os.listdir(labels_path):
  label = pd.read_csv(os.path.join(labels_path, labelname), sep=' ', header=None, index_col = None)
  label = np.asarray(label)
  label = label[:,1:] #this line should be omitted if the data only contains 4 columns
  
  if label.shape[0]<5:
    zeros = np.zeros((5-label.shape[0],4))
    label = np.concatenate([label, zeros], axis=0)
  
  if label.shape[0]>5:
    label = label[0:5,:]
  label_list.append(label)
  #print(len(label_list))


for imgname in os.listdir(images_path):
  image = Image.open(os.path.join(images_path, imgname))
  data = np.asarray(image)
  #print(data.shape)
  image_list.append(data)
  #image_list = np.array(image_list)
 # print(image_list)



tensor_image = torch.Tensor(image_list)
#print(tensor_image.shape) # transform to torch tensor
tensor_label = torch.Tensor(label_list)
print(tensor_label)

my_dataset = TensorDataset(tensor_image, tensor_label) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

#my_dataset_label = TensorDataset(tensor_label) # create your datset
#my_dataloader_label = DataLoader(my_dataset_label)
#print(len(my_dataloader))

tensor([[[0.8930, 0.8738, 0.1322, 0.1322],
         [0.8522, 0.6755, 0.1178, 0.1202],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.8438, 0.5036, 0.0577, 0.0601],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.2500, 0.9014, 0.1154, 0.1154],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        ...,

        [[0.1058, 0.4363, 0.1298, 0.1322],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.1142, 0.7776, 0.1659, 0.1659],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000

In [8]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/Mars_GAN/labels/aeolis_43_5.txt', sep=' ', header=None, index_col = None )
df = np.asarray(df)
print(df)
df = df[:,1:]

if df.shape[0]<5:
  zeros = np.zeros((5-df.shape[0],4))
  df = np.concatenate([df, zeros], axis=0)
print(df)

zeros = np.zeros((7-df.shape[0],4))
df = np.concatenate([df, zeros], axis=0)
print(df)

if df.shape[0]>5:
  df = df[0:5,:]
print(df)

[[0.         0.89302885 0.87379808 0.13221154 0.13221154]
 [0.         0.85216346 0.67548077 0.11778846 0.12019231]]
[[0.89302885 0.87379808 0.13221154 0.13221154]
 [0.85216346 0.67548077 0.11778846 0.12019231]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
[[0.89302885 0.87379808 0.13221154 0.13221154]
 [0.85216346 0.67548077 0.11778846 0.12019231]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
[[0.89302885 0.87379808 0.13221154 0.13221154]
 [0.85216346 0.67548077 0.11778846 0.12019231]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


In [68]:
class Generator(nn.Module):
    def __init__(self, g_input_dim=100, g_output_dim=416*416+20):
        super().__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    
    # forward method
    def forward(self, x, c): 
        x = torch.flatten(x)
        c = torch.flatten(c)
        print(x.shape)
        print(c.shape)
        x = torch.cat([x, c], dim=-1)
        print(x.shape)
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim=416*416+20):
        super().__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1) #input size
    
    # forward method
    def forward(self, x, c):
        x = torch.flatten(x)
        c = torch.flatten(c)
        print(x.shape)
        print(c.shape)
        x = torch.cat([x, c], dim=-1)
        x = F.leaky_relu(self.fc1(x), 0.2)
        #x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        #x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        #x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))
G = Generator().to(device)
D = Discriminator().to(device)

In [28]:
# define loss
criterion = nn.BCELoss() 
z_dim = 100
bs = 1  ## batch_size


# optimiser
lr = 0.0001 
G_optimizer = torch.optim.Adam(G.parameters(), lr = lr)
D_optimizer = torch.optim.Adam(D.parameters(), lr = lr)

In [53]:
G


Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=173076, bias=True)
)

In [66]:
D

Discriminator(
  (fc1): Linear(in_features=173076, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=2, bias=True)
)

In [86]:
def D_train(x,c):
    #-------------- Function of the discriminator training -------------------#
    D.train()
    D_optimizer.zero_grad()


    # train discriminator on real data -- assign high score (use 1 here)
    x_real, y_real = x.view(-1, 416**2), torch.ones(bs, 1)  # we are assigning the label 'real data' to the samples (don't care anymore about what number they are)
    x_real, y_real = x_real.to(device), y_real.to(device)


    D_output = D(x_real, c.to(device)) ##remember this!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #D_output = D(x_real)
   # D_output = D_output.view(1, 1)
    D_real_loss = criterion(D_output.to(device), y_real)

    # train discriminator on fake data -- assign low score (use 0 here)
    # sample vector and produce generator output
    z = torch.randn(bs, z_dim).to(device)
    x_fake, y_fake = G(z, c.to(device)), torch.zeros(bs, 1).to(device)

    
    D_output = D(x_fake.to(device), c.to(device)) ##remember this!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #D_output = D(x_fake.to(device))
    D_fake_loss = criterion(D_output, y_fake)

    # combine the losses
    D_loss = D_real_loss + D_fake_loss

    # model update 
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()  ### deprecated version of loss.detach(), basically gets access to the tensor without the computational graph attached

In [82]:
def G_train(x,c):
    #-------------- Function of the generator training -------------------#
    G.train()
    G_optimizer.zero_grad()

    # sample vector and produce generator output
    z = torch.randn(bs, z_dim).to(device)
    G_output = G(z, c.to(device))

    # obtain scores from D for the generated data
    D_output = D(G_output, c.to(device))

    # train generator to "fool" discriminator
    y = torch.ones(bs, 1).to(device)
    G_loss = criterion(D_output, y)

    # model update 
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()  ### deprecated version of loss.detach(), basically gets access to the tensor without the computational graph attached
     

In [45]:
for a, (i,j) in enumerate(my_dataloader):
  print(a)
  print(i.shape)
  print(j.shape)
  i = torch.flatten(i)
  j = torch.flatten(j)
  print(i.shape)
  print(j.shape)
  #new = torch.cat([i, j], dim=)
  #torch.set_printoptions(profile="full")
  #print(new)
  break

0
torch.Size([1, 416, 416])
torch.Size([1, 5, 4])
torch.Size([173056])
torch.Size([20])


In [87]:
import torch.nn.functional as F
n_epoch = 200 # about 40 minutes
groups = {'Loss': ['D_Loss', 'G_Loss']}
liveloss = PlotLosses(groups=groups)

for epoch in range(1, n_epoch+1):  
  D_losses, G_losses = [], []
  logs = {}

  for batch_idx, (img, labels) in enumerate(my_dataloader):
    logs['D_Loss'] = D_train(img, labels)
    logs['G_Loss'] = G_train(img, labels)
      
  liveloss.update(logs)
  liveloss.draw()

  # save every 20th epochs 
  if(np.mod(epoch, 20) == 0):
    torch.save(G.state_dict(), "./Generator_{:03d}.pth".format(epoch))

torch.Size([173056])
torch.Size([20])


ValueError: ignored